In [9]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.ndimage import gaussian_filter
from tqdm import tqdm

In [ ]:
# Parameters
timesteps = 1000
grid_size = 32  # Smaller size for better performance in 3D
num_bubbles = 3
min_radius, max_radius = 4, 4  # Minimum and maximum bubble radii
blur_sigma = 1  # Standard deviation for Gaussian blur


# Initialize bubble positions and directions in 3D
def create_3d_bubble(
    timesteps, grid_size, num_bubbles, min_radius, max_radius, blur_sigma=1
)->np.ndarray:
    bubble_positions = np.random.uniform(0, grid_size, (num_bubbles, 3))
    bubble_directions = np.random.normal(0, 1, (num_bubbles, 3))
    bubble_directions = bubble_directions / np.linalg.norm(
        bubble_directions, axis=1, keepdims=True
    )

    # Initialize random velocities for each bubble
    bubble_velocities = np.random.uniform(1, 2, (num_bubbles, 1))
    bubble_directions *= bubble_velocities  # Scale directions by velocity

    # Initialize bubble radii and growth parameters
    bubble_sizes = np.random.uniform(min_radius, max_radius, num_bubbles)
    bubble_growth_directions = np.random.choice([-1, 1], num_bubbles)
    bubble_growth_rates = np.random.uniform(0.1, 0.3, num_bubbles)

    # Initialize 3D flow
    flow = np.zeros((timesteps, grid_size, grid_size, grid_size))

    # Simulate the flow
    for t in tqdm(range(timesteps)):
        # Clear grid
        grid = np.zeros((grid_size, grid_size, grid_size))

        # Update bubble positions and enforce periodic boundary conditions
        bubble_positions += bubble_directions
        bubble_positions %= grid_size

        # Update bubble sizes
        for i in range(num_bubbles):
            bubble_sizes[i] += bubble_growth_directions[i] * bubble_growth_rates[i]
            if bubble_sizes[i] >= max_radius or bubble_sizes[i] <= min_radius:
                bubble_growth_directions[i] *= -1  # Reverse growth direction

        # Draw bubbles in the grid
        for i, pos in enumerate(bubble_positions):
            x, y, z = np.round(pos).astype(int)
            radius = int(bubble_sizes[i])
            for dx in range(-radius, radius + 1):
                for dy in range(-radius, radius + 1):
                    for dz in range(-radius, radius + 1):
                        if dx**2 + dy**2 + dz**2 <= radius**2:
                            nx, ny, nz = (
                                (x + dx) % grid_size,
                                (y + dy) % grid_size,
                                (z + dz) % grid_size,
                            )
                            grid[nx, ny, nz] = 1

        # Apply Gaussian blur
        grid = gaussian_filter(grid, sigma=blur_sigma)

        # Save grid to flow
        flow[t] = grid
    return flow


new_flow = create_3d_bubble(timesteps, grid_size, num_bubbles, min_radius, max_radius, blur_sigma)
np.save(
    "/home/l1148900/dns_project/G-LED-custom/data/3d_bubbles/bubble_0.npy",
    new_flow.astype(np.float16),
)

  3%|▎         | 33/1000 [00:00<00:03, 288.96it/s]

100%|██████████| 1000/1000 [00:05<00:00, 170.40it/s]


In [ ]:

# Create a 3D visualization of the 3D flow
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection="3d")
ax.set_box_aspect([1, 1, 1])

# Set fixed bounds for the plot
ax.set_xlim(0, grid_size)
ax.set_ylim(0, grid_size)
ax.set_zlim(0, grid_size)

# Initial frame
def plot_frame(frame, ax):
    ax.clear()
    ax.set_title(f"3D Flow Simulation: Timestep {frame}")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")

    # Set fixed bounds for consistency
    ax.set_xlim(0, grid_size)
    ax.set_ylim(0, grid_size)
    ax.set_zlim(0, grid_size)

    # Extract 3D grid for this timestep
    grid = flow[frame]
    x, y, z = np.nonzero(grid > 0.5)

    # Plot the points
    ax.scatter(x, y, z, c=grid[x, y, z], cmap="viridis", marker="o", alpha=0.8)

# Animation function
def update(frame):
    plot_frame(frame, ax)
    return [ax]

ani = FuncAnimation(fig, update, frames=timesteps, interval=100, blit=False)

video_path = "3d_flow_simulation.mp4"
ani.save(video_path, writer="ffmpeg", fps=10)
plt.close(fig)

print(f"3D video with variable bubble radius saved at: {video_path}")


In [68]:
# new_flow=np.stack([flow,flow], axis=1)
new_flow=flow.reshape(flow.shape[0],1,*flow.shape[1:])
np.save("/home/l1148900/dns_project/G-LED-custom/data/3d_bubbles/bubble_0.npy", new_flow.astype(np.float16))
np.save("/home/l1148900/dns_project/G-LED-custom/data/3d_bubbles/bubble_1.npy", new_flow.astype(np.float16))
np.save("/home/l1148900/dns_project/G-LED-custom/data/3d_bubbles/bubble_2.npy", new_flow.astype(np.float16))

In [67]:
# np.stack([flow,flow], axis=1).shape
flow.reshape(flow.shape[0],1,*flow.shape[1:]).shape

(1000, 1, 32, 32, 32)

In [44]:
flow.reshape(*flow.shape,1).shape

(1000, 24, 24, 24, 1)